In [6]:
import pandas as pd
!pip install openai
!pip install pandas
!pip install aiohttp
!pip install lxml


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/89/a9/63af38c7f42baff8251d937be91c6decfe9e4725fe16283dcee428e08d5c/lxml-5.3.0-cp39-cp39-macosx_10_9_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 4.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


First, create list of keywords necessary in the keyword.txt file.

Then, run command with the following command that will give top ten article information from 2022 to 2024 and save data to article_data.csv

In [8]:
!python async_pubmed_scraper.py --pages 1 --start 2018 --stop 2020 --output article_data

/Users/devammondal/PycharmProjects/geneKnowledgeGraphs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(

Finding PubMed article URLs for 5 keywords found in keywords.txt

Scraping initiated for 50 article URLs found from 2018 to 2020

Preview of scraped article data:

      title     abstract  ...     keywords                                       url
0  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/31157963
1  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/31292550
2  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/31093808
3  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/29988080
4  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/28925400

[5 rows x 8 columns]

Load the first five entries of the results into a pandas dataframe.

In [10]:
import pandas as pd
firstFive = pd.read_csv('/Users/devammondal/PycharmProjects/geneKnowledgeGraphs/article_data.csv')

In [15]:
firstFive[35:42]

,Unnamed: 0,title,abstract,affiliations,authors,journal,date,keywords,url
35,35,The BRCA1 and BRCA2 Genes in Early-Onset Breas...,Approximately 5-10% of breast cancers are attr...,"['1 Regenerative Medicine Cluster, Advanced Me...","Mohamed Saleem, Mohd Bazli Ghazali, Md Azlan M...",Advances in experimental medicine and biology,NO_DATE,BRCA1; BRCA2; Breast cancer; Early onset.,https://pubmed.ncbi.nlm.nih.gov/29687286
36,36,A selective BCL-XL PROTAC degrader achieves sa...,B-cell lymphoma extra large (BCL-XL) is a well...,"['1 Department of Pharmacodynamics, University...","Sajid Khan, Xuan Zhang, Dongwen Lv, Qi Zhang, ...",Nature medicine,NO_DATE,NO_KEYWORDS,https://pubmed.ncbi.nlm.nih.gov/31792461
37,37,Splicing factor mutant myelodysplastic syndrom...,The myelodysplastic syndromes (MDS) are common...,"['1 Bloodwise Molecular Haematology Unit, Nuff...","Andrea Pellagatti, Jacqueline Boultwood,",Advances in biological regulation,NO_DATE,Myelodysplastic syndromes; NF-κB signaling; R-...,https://pubmed.ncbi.nlm.nih.gov/31558432
38,38,U2AF1 mutations induce oncogenic IRAK4 isoform...,Spliceosome mutations are common in myelodyspl...,"[""1 Division of Experimental Hematology and Ca...","Molly A Smith, Gaurav S Choudhary, Andrea Pell...",Nature cell biology,NO_DATE,NO_KEYWORDS,https://pubmed.ncbi.nlm.nih.gov/31011167
39,39,Therapeutic Targeting of RNA Splicing Catalysi...,Cancer-associated mutations in genes encoding ...,"['1 Institute of Molecular and Cell Biology, A...","Jia Yi Fong, Luca Pignata, Pierre-Alexis Goy, ...",Cancer cell,2019,AML; Arginine methylation; MDS; MS023; PRMT1; ...,https://pubmed.ncbi.nlm.nih.gov/31408619
40,40,"H3B-8800, an orally available small-molecule s...",Genomic analyses of cancer have identified rec...,"['1 H3 Biomedicine Inc., Cambridge, Massachuse...","Michael Seiler, Akihide Yoshimi, Rachel Darman...",Nature medicine,NO_DATE,NO_KEYWORDS,https://pubmed.ncbi.nlm.nih.gov/29457796
41,41,T cell receptor gene therapy targeting WT1 pre...,Relapse after allogeneic hematopoietic cell tr...,"['1 Program in Immunology, Fred Hutchinson Can...","Aude G Chapuis, Daniel N Egan, Merav Bar, Thom...",Nature medicine,NO_DATE,NO_KEYWORDS,https://pubmed.ncbi.nlm.nih.gov/31235963


Goal: get RDF triples for all abstracts.